In [1]:
import numpy as np
import scipy as sp
import matplotlib as mpl
import pandas as pd
import csv
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline
from scipy import integrate
from lmfit import minimize, Parameters, Parameter, report_fit
from sklearn import datasets
from sklearn.linear_model import LinearRegression
import random
from shapely.geometry import Polygon
from shapely.geometry import Point


In [2]:
def calculate_iou(box_1, box_2):
    poly_1 = Polygon(box_1)
    poly_2 = Polygon(box_2)
    iou = poly_2.intersection(poly_1).area / poly_1.union(poly_2).area
    proportion=poly_2.area/(poly_1.union(poly_2).area)
    return (iou/proportion)

In [3]:
sizematrix=pd.read_csv('sizematrix.csv',header=None)

In [4]:
diametermatrix=sizematrix*2
diametermatrix

,0
0,2.000
1,2.018
2,2.036
3,2.054
4,2.072
...,...
2996,1992.800
2997,1994.600
2998,1996.400
2999,1998.200


In [5]:
particledata=pd.read_csv('ASR230_ETproxy_filteredGT300.csv')
particledata.rename(columns = {' xFmin':'minordiameter', ' xFmax':'majordiameter', ' Area':'AREA'}, inplace = True)
#particledata.rename(columns = {'BOUNDING RECT HEIGHT':'minordiameter', 'MAX FERET DIAMETER':'majordiameter'}, inplace = True)
particledata

,Particle#,Time,AREA,Convex Area,Perimeter,Convex Perim,xLF,minordiameter,majordiameter
0,0,0.0,332942.7,341797.5,2055.729,2037.096,673.7574,643.8961,676.7117
1,1,0.0,317470.0,324320.8,2003.469,1990.272,646.8503,617.8869,662.8671
2,2,1.0,378149.1,387097.1,2187.851,2175.869,698.0094,695.8275,713.7173
3,3,2.0,466977.2,478069.1,2439.205,2426.809,799.5746,768.8830,806.5934
4,4,2.0,319334.2,326511.2,2012.767,1989.422,648.5399,634.8884,656.7848
...,...,...,...,...,...,...,...,...,...
7425,13293,294.0,298175.7,306191.7,1947.686,1936.844,646.8503,598.5779,651.4459
7426,13294,294.0,318961.3,331497.9,2025.481,1989.567,655.3656,641.7142,660.4728
7427,13296,295.0,283169.0,290812.2,1892.568,1884.758,607.5856,600.7502,618.6400
7428,13297,296.0,329680.3,338069.2,2046.065,2030.791,682.6781,641.7142,683.9042


In [6]:
areaeqdiameter=np.array(np.sqrt(4*particledata.AREA/np.pi))
areaeqdiameter

array([651.08817511, 635.77933142, 693.88355502, ..., 600.45147068,
       647.89041904, 782.24091251])

In [7]:
x_rc=np.empty((len(areaeqdiameter),1))
for q in range (0,len(areaeqdiameter),1):
    a=(np.pi-2)/8
    b=(particledata.majordiameter[q]-particledata.minordiameter[q])
    c=((4*particledata.majordiameter[q]*particledata.minordiameter[q]+(np.pi-2)*particledata.minordiameter[q]**2)/8)-particledata.AREA[q]
    x_rc[q]=(-b+np.sqrt((b**2)-4*a*c))/(2*a)
x_rc=x_rc.flatten()
x_rc

array([521.45176055, 500.19404201, 592.76984922, ..., 507.57755375,
       470.7378716 , 604.75679405])

In [8]:
diameters=np.empty((len(x_rc),1))
for n in range (0,len(x_rc),1):
    total=int(particledata.minordiameter[n])+int(x_rc[n])
    binom=random.randint(0, total)
    if binom <= int(particledata.minordiameter[n]):
        diameters[n]=particledata.minordiameter[n]
    else: 
        diameters[n]=x_rc[n]
diameters

array([[643.8961   ],
       [617.8869   ],
       [695.8275   ],
       ...,
       [600.7502   ],
       [470.7378716],
       [764.5964   ]])

In [9]:
average=np.average(particledata.minordiameter)
average

611.2812765545087

In [10]:
indices=np.empty((len(particledata),1))
for n in range (0,len(indices),1):
    array = np.asarray(diametermatrix)
    value=diameters[n]
    idx = (np.abs(array - value)).argmin()
    indices[n]=idx
indices

array([[2247.],
       [2232.],
       [2275.],
       ...,
       [2223.],
       [2150.],
       [2314.]])

In [11]:
contactradiii=np.empty((len(indices),1))
for g in range (0,len(indices),1):
    diametermatrix2=np.array(diametermatrix)
    contactradiii[g]=diametermatrix2[int(indices[g])]
contactradiii

array([[644.6],
       [617.6],
       [695. ],
       ...,
       [601.4],
       [470. ],
       [765.2]])

In [12]:
totalelements=np.empty((3000,1))
for x in range(1,3001,1):
    with open(r'C:\Users\lfeltner\Dropbox\CSEE\ParticleSelector\UpdatedParticles\shot{}.inp'.format("{:04d}".format(x))) as p:
        for line in p:
            pass
        last_line = line
        totalelements[x-1]=int(last_line[0:4])

In [13]:
length = 5
length

5

In [14]:
width=5
width

5

In [15]:
massflowrate_kg=float(input("Mass flow rate (kg/s):"))
##  0.188997 kg/s

Mass flow rate (kg/s): 0.0945


In [16]:
peeningtime=float(input("Peening time (s):"))
## 40 s

Peening time (s): 20


In [17]:
partarea=float(input("Total Peened Part Area (mm^2):"))
## 23300 mm^2 or 49100 mm^2

Total Peened Part Area (mm^2): 49100


In [18]:
thetacenter=float(input("Impact Angle (degrees):"))

Impact Angle (degrees): 30


In [19]:
velo=float(input("Impact velocity  (mm/s):"))

Impact velocity  (mm/s): 65000


In [20]:
particlemass_kg=(length*width/partarea)*peeningtime*massflowrate_kg
particlemass_mt=particlemass_kg/1000
particlemass_kg

0.0009623217922606923

In [21]:
'''
coverage=float(input("Coverage (%): "))/100
coverage
'''

'\ncoverage=float(input("Coverage (%): "))/100\ncoverage\n'

In [22]:
jobname=input("Job Name: (No Spaces, Cannot begin with number)")
jobname

Job Name: (No Spaces, Cannot begin with number) S230_Proxy_0945_20_49100_65000_30


'S230_Proxy_0945_20_49100_65000_30'

In [23]:
striptype=input("Strip Type: (N, A, or C)")
striptype

Strip Type: (N, A, or C) A


'A'

In [24]:
'''
dimplesize=0.0439

n_impacts= int(length*width*coverage/dimplesize)
n_impacts
'''

'\ndimplesize=0.0439\n\nn_impacts= int(length*width*coverage/dimplesize)\nn_impacts\n'

In [25]:
'''
impacts=random.sample(range(0, len(particledata)-1), n_impacts)
#impacts
'''

'\nimpacts=random.sample(range(0, len(particledata)-1), n_impacts)\n#impacts\n'

In [26]:
density=7.98*10**-9
density

7.980000000000001e-09

In [27]:
rollingmass=0
m=0
impacts=np.empty((0,1))
while rollingmass < particlemass_mt:
    impacts=np.append(impacts,random.sample(range(0, len(particledata)-1), 1))
    radius=areaeqdiameter[int(impacts[m])]/2000
    particlemass=(4/3)*np.pi*density*radius**3
    m=m+1
    rollingmass=rollingmass+particlemass
impacts

array([7.388e+03, 5.636e+03, 3.970e+03, 5.371e+03, 2.505e+03, 2.262e+03,
       4.250e+02, 2.610e+03, 4.569e+03, 2.026e+03, 5.530e+02, 3.507e+03,
       1.046e+03, 2.699e+03, 4.522e+03, 6.616e+03, 2.122e+03, 3.179e+03,
       1.975e+03, 4.623e+03, 6.303e+03, 2.491e+03, 6.622e+03, 1.919e+03,
       1.444e+03, 1.408e+03, 4.610e+02, 2.618e+03, 4.920e+02, 4.451e+03,
       7.287e+03, 3.657e+03, 5.268e+03, 6.202e+03, 4.384e+03, 2.545e+03,
       4.325e+03, 7.363e+03, 1.373e+03, 6.337e+03, 6.040e+03, 3.832e+03,
       4.953e+03, 6.872e+03, 2.272e+03, 4.325e+03, 1.136e+03, 4.590e+02,
       2.625e+03, 2.218e+03, 2.092e+03, 3.752e+03, 2.888e+03, 2.202e+03,
       5.307e+03, 6.028e+03, 1.531e+03, 1.842e+03, 4.331e+03, 5.966e+03,
       4.230e+02, 5.414e+03, 6.689e+03, 2.521e+03, 6.663e+03, 8.690e+02,
       4.300e+02, 2.467e+03, 1.442e+03, 3.826e+03, 4.669e+03, 3.703e+03,
       1.540e+03, 3.223e+03, 2.368e+03, 2.630e+02, 4.894e+03, 5.153e+03,
       6.197e+03, 4.993e+03, 6.909e+03, 2.530e+03, 

In [28]:
print("{},{},{}".format(len(impacts), rollingmass, particlemass_mt))

876,9.627419283392328e-07,9.623217922606923e-07


In [29]:
density=7.98*10**-9
density

7.980000000000001e-09

In [30]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

In [31]:
particles_call=np.empty((len(impacts),1))
for x in range (0,len(impacts),1):
    particles_call[x]=find_nearest(diametermatrix,diameters[int(impacts[x])])

In [32]:
radius=areaeqdiameter[int(impacts[5])]/2000
radius

0.3213243690972288

In [33]:
#xcoord=np.empty((len(impacts),1))
#ycoord=np.empty((len(impacts),1))
#zcoord=np.empty((len(impacts),1))
xcoord=[float(random.randint(0, 5000))/1000 for _ in range(len(impacts))]
ycoord=[float(random.randint(0, 5000))/1000 for _ in range(len(impacts))]
zcoord=[float(random.randint(3000, 6000))/1000 for _ in range(len(impacts))]
theta=[np.random.normal(loc=thetacenter, scale=.1) for _ in range(len(impacts))]
phi=[np.random.normal(loc=0, scale=.1) for _ in range(len(impacts))]
#theta=[float(np.random.normal(90, sigma))/1000 for _ in range(len(impacts))]
#phi=[float(random.randint(0, 360000))/1000 for _ in range(len(impacts))]



In [34]:
header=("*Heading\n** Job name: {} Model name: {}\n** Generated by: LAF INP Generator\n*Preprint, echo=NO, model=NO, history=NO, contact=NO\n**\n** PARTS\n".format(jobname,jobname))
textheader=("x,y,AreaEqDiameter,impactdiameter\n")
f=open('{}.inp'.format(jobname),'a+')
t=open('{}.txt'.format(jobname),'a+')
f.write(header)
t.write(textheader)
for x in range (0,len(impacts),1):
    name="shot"
    radius=areaeqdiameter[int(impacts[x])]/2000
    mass=(4/3)*np.pi*density*radius**3
    #############################################################
    box=[[0,0],[0,5],[5,5],[5,0]]
    contactradius=diameters[x]/2000
    circle=Point(xcoord[x],ycoord[x]).buffer(contactradius)
    intersect=calculate_iou(box,circle)
    mass=mass*intersect
    #######################################################################################
    temp=("**\n*Part, name={}{}\n" .format(name, "{:05d}".format(x)))
    f.write(temp)
    j=int(particles_call[x])
    temp=open(r"C:\Users\lfeltner\Dropbox\CSEE\ParticleSelector\UpdatedParticles - Copy (2)\{}{}.inp".format(name, "{:02d}".format(j)),'r')
    f.write((temp.read()[22:]))
    temp="*ELSET, ELSET=E{}, GENERATE\n 1,     {} \n*Node\n      1,           0.,           0.,           0.\n*Nset, nset=shot{}-RefPt_, internal\n1, \n*Nset, nset=Set-1\n 1,\n*Rigid Body, ref node=shot{}-RefPt_, ELSET=E{}\n*Element, type=MASS, elset=Set-1_Inertia-1_\n{}, 1\n*Mass, elset=Set-1_Inertia-1_\n{}, \n".format("{:05d}".format(x),"{:04d}".format(int(totalelements[int(particles_call[x])-1])), "{:05d}".format(x),"{:05d}".format(x),"{:05d}".format(x),"{:04d}".format(int(totalelements[int(particles_call[x])-1])),'{:.20f}'.format(mass))
    f.write(temp)
    temp=("*End Part\n")
    f.write(temp)
substrate=open("{}_Subs_Part.txt".format(striptype))
f.write(substrate.read())
f.write("\n**\n**\n** ASSEMBLY\n**\n*Assembly, name={}_assembly\n**  \n*Instance, name={}_SUBS-1, part={}_SUBS\n 0,0,-1.295\n*End Instance\n".format(jobname, striptype, striptype))
for x in range (0,len(impacts),1):
    #xcoord=float(random.randint(0, 5000))/1000
    #ycoord=float(random.randint(0, 5000))/1000
    #zcoord=float(random.randint(3000, 6000))/1000
    radius=areaeqdiameter[int(impacts[x])]/2000
    #########################################################################
    adjx=xcoord[x]+((zcoord[x]-radius)*np.sin(np.radians(90-theta[x]))*np.cos(np.radians(phi[x])))
    adjy=ycoord[x]+((zcoord[x]-radius)*np.sin(np.radians(90-theta[x]))*np.sin(np.radians(phi[x])))
    adjz=(zcoord[x]-(0.65*radius))*np.cos(np.radians(90-theta[x]))
    ##adjz=(zcoord[x]+1.295-radius)*np.cos(np.radians(90-theta[x]))
    ########################################################################
    temp=("**  \n*Instance, name=shot{}, part=shot{}\n{},    {},          {}\n{},    {},          {}, {},    {},          {}, {}\n*End Instance\n" .format("{:05d}".format(x),"{:05d}".format(x), adjx, adjy, adjz, adjx, adjy, adjz, adjx, adjy+1, adjz, int(90)))
    f.write(temp)
    temp3=("{},{},{},{}\n".format(xcoord[x],ycoord[x],radius*2,float(contactradiii[int(impacts[x])]/1000)))
    t.write(temp3)
#temp=open("{}_elsets.inp".format(striptype),'r')
#f.write(temp.read())
f.write("**\n")
for x in range (0,len(impacts),1):
    temp=("*Nset, nset=RefPts, instance=shot{}\n 1,\n".format("{:05d}".format(x)))
    f.write(temp)
for x in range (0,len(impacts),1):
    temp=("*Nset, nset=SHOT{}_VeloSet, instance=shot{}\n 1,\n".format("{:05d}".format(x),"{:05d}".format(x)))
    f.write(temp)
    ##Assign velocity to each nset based on phi and theta
subssets=open("{}_Subs_Sets.txt".format(striptype))
f.write(subssets.read())
f.write("\n*End Assembly\n")
materials=open("{}_Subs_Mats.txt".format(striptype))
f.write(materials.read())
f.write("\n**\n ** PREDEFINED FIELDS \n")
for x in range (0,len(impacts),1):
    xvelo=-velo*np.sin(np.radians(90-theta[x]))*np.cos(np.radians(phi[x]))
    yvelo=-velo*np.sin(np.radians(90-theta[x]))*np.sin(np.radians(phi[x]))
    zvelo=-velo*np.cos(np.radians(90-theta[x]))
    temp=("** \n** Name: ShotVelo{}   Type: Velocity\n*Initial Conditions, type=VELOCITY\nSHOT{}_VeloSet, 1, {}\nSHOT{}_VeloSet, 2, {}\nSHOT{}_VeloSet, 3, {}\n".format("{:05d}".format(x),"{:05d}".format(x), xvelo,"{:05d}".format(x), yvelo, "{:05d}".format(x), zvelo))
    f.write(temp)
#temp=open"BCsAndSteps.inp", 'r')
#f.write(temp.read())
ints=open("{}_Subs_Ints.txt".format(striptype))
f.write(ints.read())
f.close()
t.close()
#change analytical surface to elset of all elements-rigid body definition

In [35]:
radius

0.31620518482891724

In [36]:
print('{:.20f}'.format(mass))

0.00000000100506811151


In [37]:
temp=open(r"C:\Users\lfeltner\Dropbox\CSEE\ParticleSelector\UpdatedParticles\shot2166.inp", 'r')
print(temp.read()[len(temp.read())-4:len(temp.read())])

In [38]:
temp=open(r"C:\Users\lfeltner\Dropbox\CSEE\ParticleSelector\UpdatedParticles\shot2166.inp", 'r')
data=temp.read()
data[-20:-15]

'3189,'

In [39]:
totalelements=np.empty((3000,1))
for x in range(1,3001,1):
    with open(r'C:\Users\lfeltner\Dropbox\CSEE\ParticleSelector\UpdatedParticles\shot{}.inp'.format("{:04d}".format(x))) as p:
        for line in p:
            pass
        last_line = line
        totalelements[x-1]=int(last_line[0:4])


In [40]:
totalelements[2092]

array([3211.])